# Importer les bibliothéques

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Importer dataset

In [2]:
dt=pd.read_csv("./dataCleaned.csv")

In [3]:
# dt[dt["Result_code"]==0]=1

In [4]:
dt.head()

,Unnamed: 0,team_code,Opponent_code,season,Round_code,Venue_code,Referee_code,Formation_code,Day_code,GF_rolling,GA_rolling,win_rolling,draw_rolling,lose_rolling,Result_code
0,3,0,5,2010,32,1,41,19,5,3.000000,0.666667,0.666667,0.333333,0.000000,2
1,4,0,31,2010,33,0,35,12,5,4.000000,0.666667,1.000000,0.000000,0.000000,0
2,5,0,35,2010,34,1,25,12,5,2.333333,1.000000,0.666667,0.333333,0.000000,1
3,6,0,11,2010,35,0,27,19,6,2.333333,1.666667,0.333333,0.333333,0.333333,1
4,7,0,2,2010,36,1,24,19,5,1.000000,2.000000,0.000000,0.333333,0.666667,2


In [5]:
train=dt[(dt["season"]<2022)&(dt["season"]>=2015)]

In [6]:
test=dt[dt["season"]>=2022]

In [7]:
x_train=train.drop(["Result_code"],axis=1)
y_train=train["Result_code"]
x_test=test.drop(["Result_code"],axis=1)
y_test=test["Result_code"]


In [8]:
x_train=x_train.drop(["Unnamed: 0"],axis=1)
x_test=x_test.drop(["Unnamed: 0"],axis=1)

In [9]:
x_train.shape

(5296, 13)

In [10]:
x_test.shape

(757, 13)

# Training

## Random Forest

In [11]:
rf=RandomForestClassifier(n_estimators=50,min_samples_split=10,random_state=1)

In [12]:
rf.fit(x_train,y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [13]:
predict_rf=rf.predict(x_test)

### Evaluation

#### Accuracy 

In [14]:
from sklearn.metrics import accuracy_score,classification_report

In [15]:
acc=accuracy_score(y_test,predict_rf)
print(classification_report(y_test, predict_rf))


              precision    recall  f1-score   support

           0       0.25      0.04      0.07       173
           1       0.49      0.62      0.55       292
           2       0.50      0.62      0.55       292

    accuracy                           0.48       757
   macro avg       0.41      0.42      0.39       757
weighted avg       0.44      0.48      0.44       757



In [16]:
acc

0.4848084544253633

## SVM

In [54]:
from sklearn.svm import SVC

In [55]:
svm = SVC(kernel='linear', C=1.0)
svm.fit(x_train, y_train)

SVC(kernel='linear')

In [56]:
pred_svm = svm.predict(x_test)

### Evaluation

#### Accuracy

In [57]:
acc_svm=accuracy_score(y_test,pred_svm)
print(classification_report(y_test, pred_svm))

              precision    recall  f1-score   support

           0       0.12      0.01      0.01       173
           1       0.52      0.67      0.59       292
           2       0.51      0.65      0.57       292

    accuracy                           0.51       757
   macro avg       0.39      0.44      0.39       757
weighted avg       0.43      0.51      0.45       757



In [20]:
acc_svm

0.6678082191780822

## XGBoost

In [69]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV


In [70]:
model = XGBClassifier()

In [72]:
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'n_estimators': [50, 100, 200]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)
grid_search.fit(x_train, y_train)
best_model = grid_search.best_estimator_
test_predictions = best_model.predict(x_test)
print(classification_report(y_test, test_predictions))


Fitting 3 folds for each of 27 candidates, totalling 81 fits
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       173
           1       0.58      0.49      0.53       292
           2       0.45      0.79      0.58       292

    accuracy                           0.50       757
   macro avg       0.35      0.43      0.37       757
weighted avg       0.40      0.50      0.43       757



c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

### Evaluation

#### Accuracy

In [24]:
acc_xg=accuracy_score(y_test,predXgb)
print(classification_report(y_test, predXgb))


NameError: name 'predXgb' is not defined

In [25]:
acc_xg

NameError: name 'acc_xg' is not defined

## Light GBM

In [63]:
import lightgbm as lgb

In [64]:
train_data = lgb.Dataset(x_train, label=y_train)
test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
params = {
    'objective': 'multiclass',
    'num_class': 3,  # Number of classes in the target variable
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}


In [65]:
num_round = 100
model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000283 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 211
[LightGBM] [Info] Number of data points in the train set: 5296, number of used features: 13
[LightGBM] [Info] Start training from score -1.453455
[LightGBM] [Info] Start training from score -0.958916
[LightGBM] [Info] Start training from score -0.959902


In [66]:
y_pred_prob = model.predict(x_test, num_iteration=model.best_iteration)
predGBM = np.argmax(y_pred_prob, axis=1)

### Evaluation

#### Accurracy

In [67]:
accuracy = accuracy_score(y_test, predGBM)
print(classification_report(y_test, predGBM))


              precision    recall  f1-score   support

           0       0.20      0.04      0.07       173
           1       0.50      0.62      0.55       292
           2       0.48      0.60      0.54       292

    accuracy                           0.48       757
   macro avg       0.39      0.42      0.38       757
weighted avg       0.43      0.48      0.43       757



In [68]:
accuracy

0.47820343461030385

## CatBoost

In [32]:
from catboost import CatBoostClassifier


In [33]:
cat_features = None
y_train_cat = y_train.astype(str)
y_test_cat = y_test.astype(str)

In [34]:
model = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass')

In [35]:
model.fit(x_train, y_train_cat, cat_features=cat_features, verbose=10)

0:	learn: 0.6823336	total: 137ms	remaining: 13.6s
10:	learn: 0.6303840	total: 154ms	remaining: 1.25s
20:	learn: 0.6100100	total: 172ms	remaining: 646ms
30:	learn: 0.5915295	total: 189ms	remaining: 421ms
40:	learn: 0.5787519	total: 208ms	remaining: 299ms
50:	learn: 0.5708555	total: 226ms	remaining: 217ms
60:	learn: 0.5620314	total: 242ms	remaining: 155ms
70:	learn: 0.5544864	total: 260ms	remaining: 106ms
80:	learn: 0.5449895	total: 283ms	remaining: 66.4ms
90:	learn: 0.5364507	total: 299ms	remaining: 29.6ms
99:	learn: 0.5283844	total: 313ms	remaining: 0us


In [36]:
predCat = model.predict(x_test)

### Evaluation

#### Accurracy

In [37]:
accuracy = accuracy_score(y_test_cat, predCat)
print(classification_report(y_test_cat, predCat))


              precision    recall  f1-score   support

           1       0.64      0.63      0.63       292
           2       0.63      0.64      0.64       292

    accuracy                           0.64       584
   macro avg       0.64      0.64      0.64       584
weighted avg       0.64      0.64      0.64       584



In [38]:
accuracy

0.6352739726027398